In [1]:
from django.http import StreamingHttpResponse
from django.shortcuts import render, redirect
import cv2
import sqlite3
import logging

logger = logging.getLogger(__name__)

camera = None

class VideoCamera:
    def __init__(self):
        self.video = cv2.VideoCapture(0)
        if not self.video.isOpened():
            logger.error("Failed to open camera")
        self.facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        try:
            self.recognizer = cv2.face.LBPHFaceRecognizer_create()
            self.recognizer.read('trainer.yml')
        except cv2.error as e:
            logger.error(f"Error initializing or loading recognizer: {e}")

    def __del__(self):
        self.video.release()

    def get_frame(self):
        success, img = self.video.read()
        if not success:
            logger.error("Failed to capture frame")
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.facedetect.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            id, conf = self.recognizer.predict(gray[y:y + h, x:x + w])
            profile = self.get_profile(id)

            if profile:
                text = f"ID: {profile[0]} - Name: {profile[2]}"
                cv2.putText(img, text, (x, y + h + 20), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 2)

        ret, jpeg = cv2.imencode('.jpg', img)
        if not ret:
            logger.error("Failed to encode frame to JPEG")
            return None

        return jpeg.tobytes()

    @staticmethod
    def get_profile(id):
        """Retrieve user profile by ID from the database."""
        try:
            conn = sqlite3.connect("sqlite.db")
            cursor = conn.execute("SELECT * FROM STUDENTS WHERE id=?", (id,))
            profile = cursor.fetchone()
            return profile
        except sqlite3.Error as e:
            logger.error(f"Database error: {e}")
            return None
        finally:
            conn.close()

def gen(camera):
    while True:
        frame = camera.get_frame()
        if frame is None:
            continue
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

def video_feed(request):
    global camera
    if camera is None:
        camera = VideoCamera()
    return StreamingHttpResponse(gen(camera), content_type='multipart/x-mixed-replace; boundary=frame')

def stop_camera(request):
    global camera
    if camera is not None:
        del camera
        camera = None
    return redirect('detection_index')

def index(request):
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.execute("SELECT * FROM STUDENTS")
    students = cursor.fetchall()
    conn.close()
    return render(request, 'detection/index.html', {'students': students})


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("sqlite.db")

# Create a cursor object
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS STUDENTS (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    grade TEXT NOT NULL
)
''')

# Function to add a student
def add_student(id, name, age, grade):
    cursor.execute("SELECT id FROM STUDENTS WHERE id=?", (id,))
    if cursor.fetchone():
        print(f"Error: Student with ID {id} already exists.")
    else:
        cursor.execute('''
        INSERT INTO STUDENTS (id, name, age, grade) VALUES (?, ?, ?, ?)
        ''', (id, name, age, grade))
        conn.commit()
        print(f"Student with ID {id} added successfully.")

# Add students to the database
students = [
    (1, 'John Doe', 20, 'A'),
    (2, 'Jane Smith', 22, 'B'),
    (3, 'Alice Johnson', 21, 'A'),
    (4, 'Bob Brown', 23, 'C')
]

for student in students:
    add_student(*student)

# Function to retrieve and print student profile
def get_student_profile(id):
    cursor.execute("SELECT * FROM STUDENTS WHERE id=?", (id,))
    profile = cursor.fetchone()
    if profile:
        print("ID:", profile[0])
        print("Name:", profile[1])
        print("Age:", profile[2])
        print("Grade:", profile[3])
    else:
        print("Student not found.")

# Retrieve and print student profiles
get_student_profile(1)
get_student_profile(2)
get_student_profile(3)
get_student_profile(4)

# Close the connection
conn.close()
